<a href="https://colab.research.google.com/github/prateekgulati/EIP/blob/master/Session6/EIP_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


### Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Load Data file

In [0]:
# load ascii text and covert to lowercase
import requests
data = requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt").text
raw_text = data.lower()

### Remove Punctuations

In [0]:
import re
raw_text = re.sub(r'[^\w\s]','',raw_text)
raw_text = re.sub(r'\r\n','\n',raw_text)

### Character to Int Mapping

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  154864
Total Vocab:  39


### Input Data with padding

In [8]:
# prepare the dataset of input to output pairs encoded as integers
from keras.preprocessing.sequence import pad_sequences
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
dataX=pad_sequences(dataX)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  154764


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

### Model Definition

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0722 05:05:07.734703 139985236694912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 05:05:07.773721 139985236694912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 05:05:07.780106 139985236694912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 05:05:08.090466 139985236694912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 05:05:08.100724 

### CheckPoints

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### Train Model

In [18]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

W0721 13:07:06.441768 140515360925568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
154764/154764 [==============================] - 332s 2ms/step - loss: 2.7270

Epoch 00001: loss improved from inf to 2.72704, saving model to weights-improvement-01-2.7270.hdf5
Epoch 2/100
154764/154764 [==============================] - 329s 2ms/step - loss: 2.3913

Epoch 00002: loss improved from 2.72704 to 2.39134, saving model to weights-improvement-02-2.3913.hdf5
Epoch 3/100
154764/154764 [==============================] - 330s 2ms/step - loss: 2.1535

Epoch 00003: loss improved from 2.39134 to 2.15350, saving model to weights-improvement-03-2.1535.hdf5
Epoch 4/100
154764/154764 [==============================] - 329s 2ms/step - loss: 1.9916

Epoch 00004: loss improved from 2.15350 to 1.99157, saving model to weights-improvement-04-1.9916.hdf5
Epoch 5/100
154764/154764 [==============================] - 327s 2ms/step - loss: 1.8755

Epoch 00005: loss improved from 1.99157 to 1.87552, saving model to weights-improvement-05-1.8755.hdf5
Epoch 6/100
154764/154764 [=======

In [0]:
model.save('./drive/My Drive/EIP3/Phase2/Session6-48-0.8490.hdf5.h5')

In [0]:
# load the network weights
model.load_weights('./drive/My Drive/EIP3/Phase2/Session6-48-0.8490.hdf5.h5')

In [13]:
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

W0722 05:05:13.247287 139985236694912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
154764/154764 [==============================] - 324s 2ms/step - loss: 0.6823

Epoch 00001: loss improved from inf to 0.68226, saving model to weights-improvement-01-0.6823.hdf5
Epoch 2/50
154764/154764 [==============================] - 319s 2ms/step - loss: 0.6664

Epoch 00002: loss improved from 0.68226 to 0.66639, saving model to weights-improvement-02-0.6664.hdf5
Epoch 3/50
154764/154764 [==============================] - 319s 2ms/step - loss: 0.7600

Epoch 00003: loss did not improve from 0.66639
Epoch 4/50
154764/154764 [==============================] - 318s 2ms/step - loss: 0.6709

Epoch 00004: loss did not improve from 0.66639
Epoch 5/50
154764/154764 [==============================] - 317s 2ms/step - loss: 0.6581

Epoch 00005: loss improved from 0.66639 to 0.65813, saving model to weights-improvement-05-0.6581.hdf5
Epoch 6/50
154764/154764 [==============================] - 316s 2ms/step - loss: 0.6564

Epoch 00006: loss improved from 0.65813 to 0.65642, saving mo

In [0]:
model.load_weights('weights-improvement-08-0.6413.hdf5')
model.save('./drive/My Drive/EIP3/Phase2/Session6-08-0.6413.hdf5')

### Load Weights

In [0]:
model.load_weights('./drive/My Drive/EIP3/Phase2/Session6-08-0.6413.hdf5')

### Int to char mappig

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

### Predict with seed text

In [32]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = list(dataX[start])
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" f
the goldfish kept running in her head and she had a vague sort of idea
that they must be collected "
 at an inllithet than
not loved 
the dormouse aglied she dorlouse wp another for you see a letter as she had fought in a loment she thought alice it would be offended and the orher thing alice was very uoleeiy she said to herself thats the bnok
her pef yith her head at she spoke eat mast any of them with the dreatures was said alice and alice was soon for some minutes alice shouted in the dounousans and the moral of that isthe of the lobster quadrille that she had not at last the mock turtle sea
Done.
